BerkeleyX: Data8.2x

Foundations of Data Science: Inferential Thinking by Resampling

In [ ]:
from datascience import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Lab 4: Inference and Capital Punishment, Part 2

Does instituting a death penalty for murder actually reduce the number of murders?

$$\text{murder rate for state X in year Y} = \frac{\text{number of murders in state X in year Y}}{\text{population in state X in year Y}}*100000$$

In [ ]:
# as seen in part 1:

murder_rates = Table.read_table('../../data/crime_rates.csv').select(
    'State', 'Year', 'Population', 'Murder Rate')
murder_rates.set_format("Population", NumberFormatter)

def diff_n(values, n):
    return np.array(values)[n:] - np.array(values)[:-n]

def two_year_changes(rates):
    "Return the number of increases minus the number of decreases after two years."
    return np.count_nonzero(diff_n(rates, 2) > 0) - np.count_nonzero(diff_n(rates, 2) < 0)

**Null Hypothesis**: State murder rates increase and decrease over two-year periods as if 
"increase" or "decrease" were sampled at random from a uniform distribution, like a fair coin flip.

**Alternative Hypothesis**: State murder rates are more likely to increase over two-year periods.

**Test Statistic**: The number of increases minus the number of decreases

1: The death penalty

A Natural Experiment

**Population:** All the states that had the death penalty before the 1972 abolition. This includes all US states **except** Alaska, Hawaii, Maine, Michigan, Wisconsin, and Minnesota.

**Treatment group:** The states in that population, in the year after 1972.

**Control group:** The states in that population, in the year before 1972.

**Null hypothesis:** Each state's murder rate was equally likely to be higher or lower in the treatment period than in the control period.  (Whether the murder rate increased or decreased in each state was like the flip of a fair coin.)

**Alternative hypothesis:** The murder rate was more likely to increase.

In [ ]:
non_death_penalty_states = np.array(['Alaska', 'Hawaii', 'Maine', 'Michigan', 'Wisconsin', 'Minnesota'])

def had_death_penalty_in_1971(state):
    """True if the state had the death penalty in 1971."""
    return state not in non_death_penalty_states

states = murder_rates.group('State').select('State')
death_penalty = states.with_column('Death Penalty', states.apply(had_death_penalty_in_1971, 0))
print(death_penalty)

In [ ]:
num_death_penalty_states = death_penalty.where("Death Penalty", are.equal_to(True)).num_rows
num_death_penalty_states

In [ ]:
# Question 1.3
# Assign death_penalty_murder_rates to a table with the same columns and data as murder_rates,
# but that has only the rows for states that had the death penalty in 1971
death_penalty_murder_rates = murder_rates.where(
    'State', are.not_contained_in(non_death_penalty_states))
death_penalty_murder_rates

In [ ]:
# Question 1.4. 
# Assign test_stat_72 to the value of the test statistic for the years 1971 to 1973
# using the states in death_penalty_murder_rates
good_years = death_penalty_murder_rates.where('Year', are.between_or_equal_to(1971, 1973))
up_or_down = good_years.group('State', two_year_changes).column(3)
test_stat_72 = np.count_nonzero(up_or_down > 0)- np.count_nonzero(up_or_down < 0)
print('Test statistic from 1971 to 1973:', test_stat_72)

In [ ]:
# Question 1.5. 
# Draw an empirical histogram of the statistic under the null hypothesis
# by simulating the test statistic 10,000 times
def simulate_under_null(num_chances_to_change):
    """Simulates some number changing several times, with an equal
    chance to increase or decrease.  Returns the value of your
    test statistic for these simulated changes.
    
    num_chances_to_change is the number of times the number changes.
    """
    uniform = Table().with_columns(
        "Change", make_array('Increase', 'Decrease'),
        "Chance", make_array(0.5,        0.5))
    sample = uniform.sample_from_distribution('Chance', num_chances_to_change)
    return sample.column(2).item(0) - sample.column(2).item(1)

iterations = 10_000

samples = np.empty(iterations, np.int16)
for i in range(iterations):
    samples[i] = simulate_under_null(num_death_penalty_states)

samples

In [ ]:
Table().with_column('Test statistic under null', samples).hist(
    bins=np.arange(-4, 28+2, 2), normed=None, density=True)
plt.scatter(test_stat_72, 0, color='red', s=40);
print('P-value:', np.count_nonzero(samples>=test_stat_72)/iterations)

2: Further evidence

- Select a table containing murder rates for certain states and all years,
- Choose two years and compute the observed value of the test statistic,
- Simulate the test statistic under the null hypothesis that increases and decreases are drawn uniformly at random, then
- Compare the observed difference to the empirical distribution to compute a P-value.

In [ ]:
def run_test(rates, start_year):
    """Prints out the observed test statistic and returns a P-value 
    for this statistic under the null hypothesis
    Parameters
    ----------
    rates : Table
       A table of murder rates for certain states, sorted by state and year like murder_rates
    start_year : int
       The year when the analysis starts
    """
    end_year = start_year + 2
    observed_test_statistic = sum(rates.where(
        'Year', are.between_or_equal_to(start_year, end_year)).select(
        'State', 'Murder Rate').group('State', two_year_changes).column(1))
    print('Test statistic', start_year, 'to', end_year, ':', observed_test_statistic)
    num_states = rates.group('State').num_rows
    
    samples = []
    for i in np.arange(5000):
        samples.append(simulate_under_null(num_states))
    samples = np.array(samples) # Convert from a list to an array
    
    if observed_test_statistic > 0:
        return sum(samples >= observed_test_statistic) / len(samples)
    elif observed_test_statistic < 0:
        return sum(samples <= observed_test_statistic) / len(samples)
    
run_test(death_penalty_murder_rates, 1971)

The rest of the states

In [ ]:
# Question 2.1
# Create a table called non_death_penalty_murder_rates 
#  with the same columns as murder_rates but only containing rows 
#  for the six states without the death penalty in 1971.
# Perform the same test on this table.
# Then, set reject_null_2 to whether their murder rates
#  were also more likely to increase from 1971 to 1973
non_death_penalty_murder_rates = murder_rates.where(
    'State', are.contained_in(non_death_penalty_states))

gy_2 = non_death_penalty_murder_rates.where('Year', are.between(1971, 1974))
ud_2 = gy_2.group('State', two_year_changes).column(3)
ts_2 = np.count_nonzero(ud_2 > 0)- np.count_nonzero(ud_2 < 0)
print('Test statistic from 1971 to 1973:', ts_2)

The death penalty reinstated

In [ ]:
print("Increases minus decreases from 1975 to 1977 (when the death penalty was reinstated) among death penalty states:",
      sum(death_penalty_murder_rates.where('Year', are.between_or_equal_to(1975, 1977))
                                    .group('State', two_year_changes)
                                    .column("Murder Rate two_year_changes")))
run_test(death_penalty_murder_rates, 1975)

3: Visualization

In [ ]:
# Question 3.1
# Create a table called average_murder_rates with 1 row for each year in murder_rates.
# It should have 3 columns:
#  Year
#  Death penalty states, the average murder rate of the states that had the death penalty in 1971
#  No death penalty states, the average murder rate of the other states
# average_murder_rates should be sorted in increasing order by year
average_murder_rates = Table().with_columns(
    'Year', murder_rates.group('Year').column(0),
    'Death penalty states', murder_rates.where(
        'State', are.not_contained_in(non_death_penalty_states)).group('Year', np.mean).column(3),
    'No death penalty states', murder_rates.where(
        'State', are.contained_in(non_death_penalty_states)).group('Year', np.mean).column(3))
average_murder_rates

In [ ]:
average_murder_rates.plot('Year')

In [ ]:
canada = Table.read_table('../../data/canada.csv')
murder_rates_with_canada = average_murder_rates.join("Year", canada.select("Year", "Homicide").relabeled("Homicide", "Canada"))
murder_rates_with_canada.plot('Year')